# Writer Verification Project

## Description:
This project is divided in two parts:
1. Data Pre-processing 
2. Training Model on convNET model(AlexNet)

First we will import all the necessary libraries for this project. For any python program we can get different libraries that make our tasks easier. You can get more details on each of these libraries in the documentation.

In [1]:
# !pip install opencv-python==4.1.0.25
# !pip install imutils
# !pip install pandas==0.23.4
# !pip install tensorflow==1.14.0
# !pip install keras==2.2.4
# !pip install split_folders
# !pip install matplotlib==3.0.2
# !pip install Pillow==5.4.0

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [2]:
import sys
import csv
import math
from PIL import Image
import PIL
import cv2
import os
import numpy as np
import imutils
import numpy as np
import split_folders
import pandas as pd
import keras
from keras import Model
from keras.models import Sequential
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import MaxPooling2D, Conv2D, Reshape, Dense, Input, Flatten, GlobalAveragePooling2D,Convolution2D, Activation, Dropout
from keras.applications.resnet50 import ResNet50
import matplotlib.pyplot as plt

Using TensorFlow backend.
C:\Users\muzaf\Anaconda3\envs\emmy\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\muzaf\Anaconda3\envs\emmy\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\muzaf\Anaconda3\envs\emmy\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\muzaf\Anaconda3\envs\emmy\lib\site-packag

### Data Preparation:

For this part we need to acquire pathches from the images of each author so that we can train our algorithm to recognize the work/writing pattern of the author.

For this we will use image opencv to detect contours in the image & copy them to a new file.

**In the next cell I have done some image processing tasks so that only the writings are visible for making patches.**

In [3]:
pic = cv2.imread("raw_data/1/DSC00005.jpg")
image = cv2.resize(pic,(int(pic.shape[1]/2),int(pic.shape[0]/2))) 
cv2.imshow("Orignal Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()
# Converting image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# Converting it to binary mapping
ret,thresh1 = cv2.threshold(gray,50,255,cv2.THRESH_BINARY_INV)
# Applying morphology
kernel = np.ones((5,5),np.uint8)
closing = cv2.morphologyEx(thresh1, cv2.MORPH_CLOSE, kernel)
# Resulting Image
cv2.imshow("Final Image", thresh1)
cv2.waitKey(0)
cv2.destroyAllWindows()

After we get that value we will extract the area only with some writing in it.

In [4]:
contours, hierarchy = cv2.findContours(closing, 1, 3)
idx = 0
x,y,w,h = (2000,2000,0,0)
for cnt in contours:
    idx += 1
    x_x, y_y, w_w, h_h = cv2.boundingRect(cnt)
    if x_x<x:
        x=x_x
    if y_y<y:
        y=y_y
    if x_x>w:
        w=x_x
    if y_y>h:
        h=y_y
crop_img = image[y:h, x:w]
crop_img_inv = thresh1[y:h, x:w]
cv2.imshow("Final Image", crop_img_inv)
cv2.waitKey(0)
cv2.destroyAllWindows()

Now we want to extract patches of same dimensions so we this code will do.

**NOTE**
1. We will also add a check if the intensity is less than so don't include that patch as it will not have much values in it.

In [5]:
(dimx,dimy,c) = crop_img.shape
div = 10
if crop_img.shape[0]>crop_img.shape[1]:
    patch_shape = (int(dimx/div),int(dimx/div))
    stride = (div*crop_img.shape[1]/int(dimx/div))
else:
    patch_shape = (int(dimy/div),int(dimy/div))
    stride = (div*crop_img.shape[0]/int(dimy/div))
s_path = "results"
if not os.path.exists(s_path):
    os.makedirs(s_path)
def patchify(img, patch_shape):
    img = np.ascontiguousarray(img)  # won't make a copy if not needed
    X, Y, C = img.shape
    h, w = patch_shape
    x,y = 0,0
    for i in range(0,200):
        crop = crop_img[y:y+h, x:x+w]
        crop_inv = crop_img_inv[y:y+h, x:x+w]
        x += h
        Intensity = np.average(crop_inv)
        if Intensity<6:
            continue
        if (crop.shape!=(h,w,C)):
            x = 0
            y += h
            continue
        try:
            crop = cv2.resize(crop,(224,224))
            cv2.imwrite(s_path + "/{}.jpg".format(i),crop)
        except:
            x = 0
            y += h
            continue
    
    return
patches = patchify(crop_img,patch_shape)

C:\Users\muzaf\Anaconda3\envs\emmy\lib\site-packages\numpy\lib\function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\Users\muzaf\Anaconda3\envs\emmy\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Now we will add all these functionalities to function that will do all that task. But first let us see how can we access all these images.

In [6]:
def patchify(crop_img, crop_img_inv, patch_shape,subdir,file):
    save_in = s_path+"/"+subdir
    if not os.path.exists(save_in):
        os.makedirs(save_in)
    img = np.ascontiguousarray(crop_img)  # won't make a copy if not needed
    X, Y, C = img.shape
    h, w = patch_shape
    x,y = 0,0
    for i in range(0,200):
        crop = crop_img[y:y+h, x:x+w]
        crop_inv = crop_img_inv[y:y+h, x:x+w]
        x += h
        Intensity = np.average(crop_inv)
        if Intensity<6:
            continue
        if (crop.shape!=(h,w,C)):
            x = 0
            y += h
            continue
        try:
            crop = cv2.resize(crop,(224,224))
            cv2.imwrite(save_in + "/{}_{}_{}.jpg".format(subdir,file,i),crop)
        except:
            x = 0
            y += h
            continue
    
    return

In [7]:
def patches_extraction(path,s_path):
    for subdir in os.listdir(path):
        print("The directory name is: ",subdir)
        name = 0
        for file in os.listdir(path+"/"+ subdir):
            print("The file name is: ",file)
            name += 1
            if not file.endswith(".jpg"):
                continue
            pic = cv2.imread(path+"/"+ subdir+"/"+file)
            image = cv2.resize(pic,(int(pic.shape[1]/2),int(pic.shape[0]/2))) 
            # cv2.imshow("Orignal Image", image)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
            # Converting image to grayscale
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            # Converting it to binary mapping
            ret,thresh1 = cv2.threshold(gray,50,255,cv2.THRESH_BINARY_INV)
            # Applying morphology
            kernel = np.ones((5,5),np.uint8)
            closing = cv2.morphologyEx(thresh1, cv2.MORPH_CLOSE, kernel)
            contours, hierarchy = cv2.findContours(closing, 1, 3)
            idx = 0
            x,y,w,h = (2000,2000,0,0)
            for cnt in contours:
                idx += 1
                x_x, y_y, w_w, h_h = cv2.boundingRect(cnt)
                if x_x<x:
                    x=x_x
                if y_y<y:
                    y=y_y
                if x_x>w:
                    w=x_x
                if y_y>h:
                    h=y_y
            crop_img = image[y:h, x:w]
            crop_img_inv = thresh1[y:h, x:w]
            (dimx,dimy,c) = crop_img.shape
            div = 10
            if crop_img.shape[0]>crop_img.shape[1]:
                patch_shape = (int(dimx/div),int(dimx/div))
            else:
                patch_shape = (int(dimy/div),int(dimy/div))
            patches = patchify(crop_img, crop_img_inv,patch_shape,subdir,name)
    return 0

Now we will gonna run this on the whole data set.

In [8]:
path = ("C:/Users/muzaf/Documents/GitHub/Text_detection/raw_data")
s_path = ("C:/Users/muzaf/Documents/GitHub/Text_detection/processed_data")
patches_extraction(path,s_path)


The directory name is:  1
The file name is:  DSC00005.jpg
The file name is:  DSC00175.jpg
The directory name is:  2
The file name is:  DSC00004.jpg
The file name is:  DSC00005.jpg
The file name is:  DSC00025.jpg
The directory name is:  3
The file name is:  DSC00004.jpg
The file name is:  DSC00006.jpg
The file name is:  DSC00353.jpg
The directory name is:  4
The file name is:  DSC00003.jpg
The file name is:  DSC00005.jpg
The directory name is:  5
The file name is:  DSC00004.jpg
The file name is:  DSC00163.jpg
The directory name is:  6
The file name is:  DSC00005.jpg
The file name is:  DSC00156.jpg
The directory name is:  7
The file name is:  DSC00005.jpg
The file name is:  DSC00006.jpg
The file name is:  DSC00062.jpg
The directory name is:  8
The file name is:  DSC00005.jpg
The file name is:  DSC00634.jpg


0

We will now split our data into train & test dataset. We will use the library "https://pypi.org/project/split-folders/".

In [9]:
s_path = ("C:/Users/muzaf/Documents/GitHub/Text_detection/processed_data")
split_path = ("C:/Users/muzaf/Documents/GitHub/Text_detection/split_data")
if not os.path.exists(split_path):
        os.makedirs(split_path)
split_folders.ratio(s_path, output=split_path, seed=1337, ratio=(.8, .1, .1))

Copying files: 1473 files [00:08, 174.11 files/s]


Now our data is all set for training! :) We will now make the model.
In this cell we initialize some parameters that will be given to our model for training.

In [10]:


train_data_path = r'C:\Users\muzaf\Documents\GitHub\Text_detection\split_data\train'
validation_data_path = r'C:\Users\muzaf\Documents\GitHub\Text_detection\split_data\val'
test_data_path = r'C:\Users\muzaf\Documents\GitHub\Text_detection\split_data\test'

"""
Parameters
"""
epochs = 10
lr = 0.0004

'''
Call Backs
'''
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("weights.{epoch:02d}-{val_loss:.2f}.hdf5",
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

## AlexNet Model

Now as discussed in paper they used an AlexNet model for training so here is the model. You can compare the model summary with that in the paper & it is similar.

In [11]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np
np.random.seed(1000)
#Instantiate an empty model
model = Sequential()

# 1st Convolutional Layer
model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11), strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

# 2nd Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

# 3rd Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))

# 4th Convolutional Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))

# 5th Convolutional Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Max Pooling
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))

# Passing it to a Fully Connected layer
model.add(Flatten())
# 1st Fully Connected Layer
model.add(Dense(4096, input_shape=(224*224*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.4))

# 2nd Fully Connected Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))

# 3rd Fully Connected Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))

# Output Layer
model.add(Dense(8))
model.add(Activation('softmax'))

model.summary()

# Compile the model
model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=['accuracy'])






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 54, 54, 96)        34944     
_________________________________________________________________
activation_1 (Activation)    (None, 54, 54, 96)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 17, 256)       2973952   
_________________________________________________________________
activation_2 (Activation)    (None, 17, 17, 256)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 256)         0         
______________________

Now we will initialize our generators. Generators help us get data as in from all the different folders

In [12]:
train_datagen = ImageDataGenerator(
    rescale=1./1,
    horizontal_flip=False)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_path,
    target_size=(224, 224),
    batch_size=6,
    class_mode="categorical")

validation_generator = test_datagen.flow_from_directory(
    validation_data_path,
    target_size=(224, 224),
    batch_size=6,
    class_mode="categorical")

test_generator = test_datagen.flow_from_directory(
    test_data_path,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=42
)

Found 1411 images belonging to 8 classes.
Found 323 images belonging to 8 classes.
Found 356 images belonging to 8 classes.


Now to find the step size we use our generators & batch size.

In [13]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

## Training

Now we will start training!

In [14]:
import sys
from PIL import Image
sys.modules['Image'] = Image 

In [16]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=3,
                    callbacks=[checkpoint]
)

Epoch 1/3
235/235 [==============================] - 142s 605ms/step - loss: 13.5920 - acc: 0.1567 - val_loss: 13.7283 - val_acc: 0.1483

Epoch 00001: val_loss improved from inf to 13.72835, saving model to weights.01-13.73.hdf5
Epoch 2/3
235/235 [==============================] - 144s 612ms/step - loss: 13.5460 - acc: 0.1596 - val_loss: 13.6775 - val_acc: 0.1514

Epoch 00002: val_loss improved from 13.72835 to 13.67750, saving model to weights.02-13.68.hdf5
Epoch 3/3
235/235 [==============================] - 143s 608ms/step - loss: 13.5917 - acc: 0.1567 - val_loss: 13.4233 - val_acc: 0.1672

Epoch 00003: val_loss improved from 13.67750 to 13.42327, saving model to weights.03-13.42.hdf5


## Evaluating the model

Since we are evaluating the model, we should treat the validation set as if it was the test set. So we should sample the images in the validation set exactly once(if you are planning to evaluate, you need to change the batch size of the valid generator to 1 or something that exactly divides the total num of samples in validation set), but the order doesn’t matter so let “shuffle” be True as it was earlier.

In [17]:
model.evaluate_generator(generator=validation_generator,
steps=STEP_SIZE_VALID)

[12.965660381016296, 0.19558360115022688]

## Predicting from the model

You need to reset the test_generator before whenever you call the predict_generator. This is important, if you forget to reset the test_generator you will get outputs in a weird order.

In [18]:
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)

356/356 [==============================] - 6s 16ms/step


Now predicted_class_indices has the predicted labels, but you can’t simply tell what the predictions are, because all you can see is numbers like 0,1,4,1,0,6…
You need to map the predicted labels with their unique ids such as filenames to find out what you predicted for which image.

In [19]:
predicted_class_indices = np.argmax(pred,axis=1)

In [20]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
labels

{0: '1', 1: '2', 2: '3', 3: '4', 4: '5', 5: '6', 6: '7', 7: '8'}

Finally we save the results as a csv file.

In [21]:
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.to_csv("results.csv",index=False)